# Coding a neural network from scratch

### Roadmap

*****

##### 1) Relire le code  de Vitali
##### 2) Recoder chacune des fonctions
##### 3) Assembler 
##### 4) Tweaker et améliorer

In [89]:
import numpy as np

In [91]:
test = np.zeros((5,5))
test2 = np.array(([3,3],[3,3],[3,3]))

In [57]:
def sigmoid(Z):
    cache = Z
    A = 1 / (1 + np.exp(-Z))
    return A, cache

def relu(Z):
    cache = Z
    A = np.maximum(0,Z)
    return A, cache

In [94]:
def initialize(layers_dims): #ex argument : [12288, 5,5,5,1]
    
    L = len(layers_dims)
    
    params = {}
    
    for i in range(1, L):
        params['W' + str(i)] = np.random.randn(layers_dims[i], layers_dims[i-1]) #  / np.sqrt(layer_dims[l-1]) #*0.01
        params['b' + str(i)] = np.zeros((layers_dims[i]))
    
    return params

In [95]:
np.random.seed(1)
initialize([2,4,1])

{'W1': array([[ 1.62434536, -0.61175641],
        [-0.52817175, -1.07296862],
        [ 0.86540763, -2.3015387 ],
        [ 1.74481176, -0.7612069 ]]),
 'b1': array([0., 0., 0., 0.]),
 'W2': array([[ 0.3190391 , -0.24937038,  1.46210794, -2.06014071]]),
 'b2': array([0.])}

In [100]:
def linear_forward(A,W,b):
    cache = A,W,b
    Z = np.dot(W,A) + b
    return Z, cache

In [101]:
def linear_activation_forward(A_prev, W, b, activation): #type(activation) == string
    
    Z, cache = linear_forward(A_prev, W, b)    
    
    if activation == "relu":
        A = relu(Z)
    elif activation == "sigmoid":
        A = sigmoid(Z)
    else:
        raise Exception(f'{activation} function is not known'.format(activation))

    return A, cache

In [111]:
def L_model(X, params):
    
    L = len(params // 2)
    caches = []
    
    A = X
    
    # L-1 premières couches
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, params['W' + str(l)], params['b' + str(l)], activation='relu')
        caches.append(cache)
    
    # L-ième couche
    A, cache = linear_activation_forward(A, params['W' + str(L)], params['b' + str(L)], activation = 'sigmoid')
    caches.append(cache)
    
    Yhat = A
    
    return Yhat, caches
    # caches est une liste qui contient L tupples, chacun contenant a) la 
    # matrice de poids pour une couche donnée et b) le vecteur de biais

In [112]:
#def loss(Y,Yhat):
#    loss = - (Y * np.log(Yhat)   +   (1 - Y) * np.log(1 - Yhat))
#    return loss

In [117]:
#def cost(X,Y, ):
#    total_loss = 0
#    
#    total += loss(Y)
#    
#    mycost = 1 / (nb exemples) * (somme des losses)
#    

In [121]:
def compute_cost(Yhat,Y):
    
    m = Y.shape[1]
    
    cost = (- 1 / m) * (np.dot(Y,np.log(Yhat).T) + np.dot((1 - Y),np.log(1 - Yhat).T))
    #moyenne des losses sous forme vectorielle. Renvoie un array.
    
    cost = np.squeeze(cost) #retire un 'niveau' de crochets
    
    return cost

In [ ]:
def linear_backward()

In [ ]:
np